In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
raw_df = pd.read_csv('Generalization results - Generalization results.tsv',sep = '\t')
raw_df.shape

(582, 3)

In [3]:
raw_df.head()

,Training dataset,Method,Evaluation results (Model accuracy F1 Lexion accuracy F1)
0,nrc_joy,ffn,"nrc_joy , 0.769 , 0.763 , 0.736 , 0.733"
1,nrc_joy,ffn,"song_joy , 0.633 , 0.622 , 0.621 , 0.596"
2,nrc_joy,ffn,"dialog_joy , 0.656 , 0.631 , 0.614 , 0.615"
3,nrc_joy,ffn,"friends_joy , 0.631 , 0.589 , 0.632 , 0.632"
4,nrc_fear,ffn,"nrc_fear , 0.765 , 0.731 , 0.736 , 0.739"


In [4]:
def cleanScore(x):
    if "," in x:
        x = x.split(",")
        
        return x
    else:
        x = x.split(" ")
        x = [y for y in x if y!= '']
        x = x[1:]
        
        return x

In [43]:
new_df = raw_df.copy()
new_df.columns = ['trainData','method','outputs']
# df = df[~pd.isna(df.outputs)]  # Removing rows with missing scores
new_df['scores'] = new_df.outputs.apply(lambda x : cleanScore(x))

scores = new_df.scores.apply(pd.Series)
scores.columns = ['testData','modelAcc','modelF1','lexiconAcc','lexiconF1']

new_df = pd.concat([new_df, scores],axis = 1)
new_df = new_df.filter(['trainData','method','testData','modelAcc','modelF1','lexiconAcc','lexiconF1'], axis = 1)

new_df.head()

,trainData,method,testData,modelAcc,modelF1,lexiconAcc,lexiconF1
0,nrc_joy,ffn,nrc_joy,0.769,0.763,0.736,0.733
1,nrc_joy,ffn,song_joy,0.633,0.622,0.621,0.596
2,nrc_joy,ffn,dialog_joy,0.656,0.631,0.614,0.615
3,nrc_joy,ffn,friends_joy,0.631,0.589,0.632,0.632
4,nrc_fear,ffn,nrc_fear,0.765,0.731,0.736,0.739


In [44]:
df = new_df.copy()
df.to_csv('new_df.csv')

In [45]:
df['trainData'] = df.trainData.apply(lambda x : x.strip())
df['method'] = df.method.apply(lambda x : x.strip())
df['testData'] = df.testData.apply(lambda x : x.strip())
df['modelAcc'] = df.modelAcc.apply(lambda x : float(x))
df['modelF1'] = df.modelF1.apply(lambda x : float(x))
df['lexiconAcc'] = df.lexiconAcc.apply(lambda x : float(x))
df['lexiconF1'] = df.lexiconF1.apply(lambda x : float(x))

In [46]:
df['Within'] = df.apply(lambda x : True if x.testData == x.trainData else False, axis = 1)

In [47]:
df.head()

,trainData,method,testData,modelAcc,modelF1,lexiconAcc,lexiconF1,Within
0,nrc_joy,ffn,nrc_joy,0.769,0.763,0.736,0.733,True
1,nrc_joy,ffn,song_joy,0.633,0.622,0.621,0.596,False
2,nrc_joy,ffn,dialog_joy,0.656,0.631,0.614,0.615,False
3,nrc_joy,ffn,friends_joy,0.631,0.589,0.632,0.632,False
4,nrc_fear,ffn,nrc_fear,0.765,0.731,0.736,0.739,True


In [49]:
df.method.value_counts()

D_BERT+mask                61
BERT+ps                    61
BERT+mask                  61
D_BERT+ps                  61
diversityLSTM+attention    61
NN+DeepShap                61
svm                        56
ffn                        56
BERT+deep                  52
D_BERT+deep                52
Name: method, dtype: int64

In [50]:
df.trainData.value_counts()

nrc_joy                   98
amazon_finefood_subset    90
amazon_toys_subset        90
empathy                   72
yelp_subset               72
nrc_anger                 40
nrc_fear                  40
nrc_surprise              40
nrc_sadness               40
Name: trainData, dtype: int64

In [57]:
df.to_csv('generalization_results.csv')

## In sample Performance

In [51]:
sub = df[df.Within == True]

In [52]:
sub.head()

,trainData,method,testData,modelAcc,modelF1,lexiconAcc,lexiconF1,Within
0,nrc_joy,ffn,nrc_joy,0.769,0.763,0.736,0.733,True
4,nrc_fear,ffn,nrc_fear,0.765,0.731,0.736,0.739,True
8,nrc_anger,ffn,nrc_anger,0.652,0.542,0.691,0.706,True
12,nrc_sadness,ffn,nrc_sadness,0.670,0.597,0.669,0.671,True
16,nrc_surprise,ffn,nrc_surprise,0.717,0.676,0.666,0.671,True


In [53]:
def getStats(x):
    
    output = []
    
    for i in ['modelAcc','modelF1','lexiconAcc','lexiconF1']:
        output += [ np.round(np.mean(x[i]),2), np.round(np.std(x[i]),2) ]
    
    return output

output = sub.groupby(['method']).apply(lambda x : getStats(x))
output = output.reset_index()

columnList = []

for i in ['modelAcc','modelF1','lexiconAcc','lexiconF1']:
    for j in ['mean','std']:
        columnList.append(f'{i}_{j}')

a = output[0].apply(pd.Series)
a.columns = columnList


output = pd.concat([output, a],axis = 1)

In [54]:
for i in range(len(output)):
    
    x = output.iloc[i]
    
    print(f"{x.method} & {x.modelAcc_mean}+-{x.modelAcc_std} & {x.modelF1_mean}+-{x.modelF1_std} & {x.lexiconAcc_mean}+-{x.lexiconAcc_std} & {x.lexiconF1_mean}+-{x.lexiconF1_std} \\")
    
    
    

BERT+deep & 0.79+-0.09 & 0.78+-0.1 & 0.53+-0.03 & 0.46+-0.1 \
BERT+mask & 0.81+-0.1 & 0.8+-0.11 & 0.72+-0.07 & 0.71+-0.07 \
BERT+ps & 0.81+-0.1 & 0.8+-0.11 & 0.75+-0.09 & 0.75+-0.09 \
D_BERT+deep & 0.78+-0.09 & 0.77+-0.11 & 0.57+-0.04 & 0.52+-0.09 \
D_BERT+mask & 0.8+-0.1 & 0.79+-0.12 & 0.73+-0.09 & 0.72+-0.1 \
D_BERT+ps & 0.8+-0.1 & 0.79+-0.12 & 0.76+-0.09 & 0.76+-0.09 \
NN+DeepShap & 0.75+-0.09 & 0.73+-0.12 & 0.65+-0.08 & 0.58+-0.12 \
diversityLSTM+attention & 0.86+-0.13 & 0.81+-0.25 & 0.74+-0.09 & 0.73+-0.1 \
ffn & 0.79+-0.1 & 0.76+-0.15 & 0.77+-0.1 & 0.78+-0.12 \
svm & 0.8+-0.09 & 0.8+-0.11 & 0.79+-0.09 & 0.8+-0.11 \


### Out of Sample Performance

In [55]:
sub = df[df.Within == False]

def getStats(x):
    
    output = []
    
    for i in ['modelAcc','modelF1','lexiconAcc','lexiconF1']:
        output += [ np.round(np.mean(x[i]),2), np.round(np.std(x[i]),2) ]
    
    return output

output = sub.groupby(['method']).apply(lambda x : getStats(x))
output = output.reset_index()

columnList = []

for i in ['modelAcc','modelF1','lexiconAcc','lexiconF1']:
    for j in ['mean','std']:
        columnList.append(f'{i}_{j}')

a = output[0].apply(pd.Series)
a.columns = columnList


output = pd.concat([output, a],axis = 1)

In [56]:
for i in range(len(output)):
    
    x = output.iloc[i]
    
    print(f"{x.method} & {x.modelAcc_mean}+-{x.modelAcc_std} & {x.modelF1_mean}+-{x.modelF1_std} & {x.lexiconAcc_mean}+-{x.lexiconAcc_std} & {x.lexiconF1_mean}+-{x.lexiconF1_std} \\  \hline")
    
    

BERT+deep & 0.67+-0.11 & 0.63+-0.16 & 0.53+-0.04 & 0.49+-0.08 \  \hline
BERT+mask & 0.69+-0.12 & 0.64+-0.17 & 0.6+-0.07 & 0.59+-0.09 \  \hline
BERT+ps & 0.69+-0.12 & 0.64+-0.17 & 0.62+-0.09 & 0.61+-0.11 \  \hline
D_BERT+deep & 0.68+-0.11 & 0.65+-0.15 & 0.55+-0.05 & 0.51+-0.09 \  \hline
D_BERT+mask & 0.69+-0.12 & 0.66+-0.16 & 0.61+-0.09 & 0.61+-0.11 \  \hline
D_BERT+ps & 0.69+-0.12 & 0.66+-0.16 & 0.64+-0.09 & 0.63+-0.09 \  \hline
NN+DeepShap & 0.64+-0.09 & 0.6+-0.15 & 0.58+-0.06 & 0.52+-0.1 \  \hline
diversityLSTM+attention & 0.62+-0.1 & 0.56+-0.19 & 0.6+-0.1 & 0.58+-0.12 \  \hline
ffn & 0.66+-0.12 & 0.63+-0.21 & 0.66+-0.12 & 0.66+-0.15 \  \hline
svm & 0.68+-0.11 & 0.68+-0.14 & 0.68+-0.11 & 0.68+-0.14 \  \hline
